<a href="https://colab.research.google.com/github/prof-rossetti/intro-to-python/blob/main/notes/python/packages/plotly/Plotting_Trends_with_Plotly_Express.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fetch Stock Data

From AlphaVantage API.



In [2]:
from getpass import getpass

API_KEY = getpass("Please input your API key:") or "demo"

Please input your API key:··········


In [3]:
from pandas import read_csv

# see: https://www.alphavantage.co/documentation/#dailyadj
# the datatype=csv URL param gives us data in CSV format, which may be easier to work with
# example: https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=demo&datatype=json

symbol = "NFLX"
request_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={API_KEY}&datatype=csv"
prices_df = read_csv(request_url)
prices_df.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2023-06-15,444.10,448.65,439.5000,445.27,445.27,7099915,0.0,1.0
1,2023-06-14,434.99,447.33,433.5000,440.86,440.86,8413089,0.0,1.0
2,2023-06-13,430.01,437.27,428.4000,435.73,435.73,7820535,0.0,1.0
3,2023-06-12,418.83,424.70,416.5669,423.97,423.97,5967712,0.0,1.0
4,2023-06-09,424.50,425.90,414.7600,420.02,420.02,12372827,0.0,1.0


## Basic Chart

In [4]:
import plotly.express as px

px.line(prices_df, x="timestamp", y="adjusted_close", title=f"Adjusted Closing Prices for {symbol}")

## Chart with Trendline

The plotly `line` function doesn't have a trend line, but the `scatter` function does. It requires both x and y series to be numeric, which is why we either create a dummy column of x time series values (using a range approach).

https://plotly.com/python/linear-fits/

### Ordinary Least Squares (OLS) Trend

In [5]:
import plotly.express as px

chart_df = prices_df.copy()
chart_df.sort_values(by=["timestamp"], ascending=True, inplace=True) # sort ascending for time series assignment
chart_df["time_series"] = range(1, len(chart_df)+1) # create numeric time series column

fig = px.scatter(chart_df, x="time_series", y="adjusted_close",
           trendline="ols", trendline_color_override="red",
           title=f"Adjusted Closing Prices for {symbol}",
           labels={"x": "Date", "y":"Stock Price (USD)"}, height=450
)
fig.show()

In [6]:
results = px.get_trendline_results(fig)
print(results.px_fit_results.iloc[0].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     20.83
Date:                Fri, 16 Jun 2023   Prob (F-statistic):           1.45e-05
Time:                        14:29:57   Log-Likelihood:                -482.86
No. Observations:                 100   AIC:                             969.7
Df Residuals:                      98   BIC:                             974.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        321.1336      6.158     52.145      0.0

### Lowess Trend

Locally Weighted Scatterplot Smoothing (LOWESS)

In [7]:
px.scatter(chart_df, x="time_series", y="adjusted_close",
           trendline="lowess", trendline_color_override="red",
           title=f"Adjusted Closing Prices for {symbol}",
           labels={"x": "Date", "y":"Stock Price (USD)"}, height=450
)

### Rolling Window Averages

In [8]:
window = 50
px.scatter(chart_df, x="time_series", y="adjusted_close",
           trendline="rolling", trendline_options=dict(window=window),
            trendline_color_override="red",
           title=f"Adjusted Closing Prices for {symbol} (EMA-{window})",
           labels={"x": "Date", "y":"Stock Price (USD)"}, height=450
)

### Exponential Moving Averages

In [9]:
halflife = 10
px.scatter(chart_df, x="time_series", y="adjusted_close",
            trendline="ewm", trendline_options=dict(halflife=halflife),
            trendline_color_override="red",
            title=f"Adjusted Closing Prices for {symbol} (EWM-{halflife})",
            labels={"x": "Date", "y":"Stock Price (USD)"}, height=450
)

# Custom Trends

We can leverage pandas to create our own calculated trend columns, and chart them.

Rolling Window Averages:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

Exponential Weighted Moving Averages:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html

In [11]:
trends_df = prices_df.copy()
trends_df.sort_values(by=["timestamp"], inplace=True) # sort for proper rolling order

trends_df["ma_50"] = trends_df["adjusted_close"].rolling(window=50).mean()

trends_df["ema_50"] = trends_df["adjusted_close"].ewm(span=50, min_periods=0, adjust=False, ignore_na=False).mean()

px.line(trends_df, x="timestamp", y=["close", "ma_50", "ema_50"],
        title=f"Adjusted Closing Prices for {symbol}",
        color_discrete_map={
                "close": "royalblue",
                "ma_50": "orange",
                "ema_50":"yellow"
            }
)

In [ ]:
#trends_df['adjusted_close'].rolling(window=3).mean()

In [ ]:
#trends_df["adjusted_close"].ewm(span=50, min_periods=0, adjust=False, ignore_na=False).mean()

In [17]:
trends_df[["adjusted_close", "ma_50", "ema_50"]]

,adjusted_close,ma_50,ema_50
99,363.83,NaN,363.830000
98,367.96,NaN,363.991961
97,364.87,NaN,364.026394
96,360.77,NaN,363.898692
95,353.11,NaN,363.475606
...,...,...,...
4,420.02,350.0950,359.017924
3,423.97,351.8058,361.565064
2,435.73,353.6108,364.473493
1,440.86,355.4624,367.469042
